In [11]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import math as m
import soundfile as sf

path_to_folder = os.getcwd()
path_to_voice = '\\LibriSpeech\\train-clean-100'
path_to_plane = '\\recordings\\'

In [12]:
def random_voice_file():
    path = path_to_folder+'\\LibriSpeech\\train-clean-100'
    dirr = os.listdir(path)
    dir1 = random.sample(dirr,1)[0]
    path = path + "\\" + dir1
    dir2 = os.listdir(path)
    dir2 = random.sample(dir2,1)[0]
    path = path + "\\" + dir2
    file = os.listdir(path)
    file = random.sample(file,1)[0]
    while file[-1] != 'c':
        file = os.listdir(path)
        file = random.sample(dir2,1)[0]
    path = path + "\\" + file
    return(path,file)

In [13]:
def random_plane_file():
    n = random.randint(0,24)
    return(path_to_folder+path_to_plane+"record_"+str(n)+".wav")

In [14]:
path,file = random_voice_file()
print(path,file[:-5])
plane = random_plane_file()
print(plane)

d:\Utilisateurs\ENAC\projet_AIRBUS\LibriSpeech\train-clean-100\1263\141777\1263-141777-0034.flac 1263-141777-0034
d:\Utilisateurs\ENAC\projet_AIRBUS\recordings\record_15.wav


In [15]:
def get_time_steps(lendata,samplerate):
    return(np.linspace(0,int(lendata/samplerate),lendata))

In [16]:
import soundfile as sf
datavoice, sampleratevoice = sf.read(path)
dataplane, samplerateplane = sf.read(random_plane_file())
print(len(dataplane),len(datavoice))

3932160 232560


In [17]:
#Les enregistrements d'avions et de voix n'ont pas le même sample rate. Afin de pouvoir les additionner facilement, on peut
#modifier le son avec le sample rate le plus petit, en considérant l'intensité constante entre les points 
#effectivement enregistrés

def same_sample_rate(voice,sr_voice,plane,sr_plane):

    if sr_voice>sr_plane:
        voice_adjusted = voice
        pas = int(sr_voice/sr_plane)
        plane_adjusted = []
        for t in range(len(plane)):
            plane_adjusted += [plane[t] for p in range(pas)]
    else:
        plane_adjusted = plane
        pas = int(sr_plane/sr_voice)
        for t in range(len(voice)):
            voice_adjusted += [voice[t] for p in range(pas)]
    return (np.array(voice_adjusted),np.array(plane_adjusted),max([sr_voice,sr_plane]))

In [18]:
def gen_samples(plane,voice,debut): #superpose deux sons de meme samplerate

    percent_voice = len(voice)/len(plane) * 100
    if percent_voice > 100-debut:
        debut_effectif = 99 - debut #on decale si ca fait trop

    else:
        debut_effectif = debut
    plane = np.array(plane)
    voice = np.array(voice)

    len_duration = len(voice)

    t_debut = int(debut/100 * len(plane))
    t_fin = t_debut + len_duration
    # print(len(res),len_duration)
    res = np.add(plane[t_debut : t_fin],voice)
    
    return voice,plane[t_debut : t_fin],np.add(plane[t_debut : t_fin],voice),debut_effectif

In [19]:
def generate_database(nb):
    for i in range(nb):
        #on selectionne les fichiers
        path_voice,file = random_voice_file()
        path_plane = random_plane_file()
        
        #on les ouvre
        datavoice, sr_voice = sf.read(path_voice)
        dataplane, sr_plane = sf.read(path_plane)

        #on les met a la meme frequence d'echantillonage
        plane,voice,sr = same_sample_rate(dataplane,sr_plane,datavoice,sr_voice)

        percent_voice = int(len(voice)/len(plane) * 100)+1
        if percent_voice<100:
            debut = random.randint(0,100-percent_voice) # en pourcent de l'extrait
            #on les superpose à partir de debut
            s_voice,s_plane,s_both,debut_effectif = gen_samples(plane,voice,debut) #il y aura 10% de voix
            
            name_file_svoice =  "db\\voice\\sample_v_{}.wav".format(i)
            sf.write(name_file_svoice, s_voice, sr)

            name_file_splane = "db\\plane\\sample_p_{}.wav".format(i)
            sf.write(name_file_splane, s_plane, sr)

            name_file_sboth = "db\\both\\sample_b_{}.wav".format(i)
            sf.write(name_file_sboth, s_both, sr)

    return True

In [20]:
generate_database(50)

KeyboardInterrupt: 